In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%load_ext autoreload
%autoreload 2

W1120 22:59:05.611985 140183612737344 def_function.py:474] 5 out of the last 5 calls to <function _moco_training_step_inner at 0x7f7e344a3bf8> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1120 22:59:05.893208 140183612737344 def_function.py:474] 6 out of the last 6 calls to <function _moco_training_step_inner at 0x7f7e344a3bf8> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnece

In [ ]:
tf.random.normal(x.shape, dtype='float32')

In [ ]:
x + tf.random.normal(x.shape, dtype='float32')

In [7]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
          train_loss.result(),
          train_accuracy.result()*100,
          test_loss.result(),
          test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

In [14]:
C = 3
q = tf.random.normal([1, C])
q = tf.tile(q, [2, 1])
k = tf.random.normal([1, C])
k = tf.tile(k, [2, 1])

In [17]:
tf.matmul(tf.reshape(q, [2, 1, C]), tf.reshape(k, [2, C, 1]))[0]

TensorShape([2, 1, 1])

In [16]:
tf.matmul(q, tf.transpose(k))

<tf.Tensor: id=95, shape=(2, 2), dtype=float32, numpy=
array([[1.7457302, 1.7457302],
       [1.7457302, 1.7457302]], dtype=float32)>